In [1]:
%cd drive/My Drive/LLM


/content/drive/My Drive/LLM


In [ ]:
!git clone https://github.com/SatwikReddySripathi/MediRAG.git

Cloning into 'MediRAG'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 71 (delta 21), reused 68 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 11.53 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (21/21), done.


Creating the DB - FAISS

In [2]:
!pip install langchain
!pip install transformers
!pip install pydantic
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.2 MB/s eta 0:00:00


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 875.8 kB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [4]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import json

# Load preprocessed data
#the path in local : datasets/pqaa_documents.json
with open("/content/drive/MyDrive/LLM/MediRAG/datasets/pqaa_documents.json", "r") as f:
    documents = json.load(f)

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vectorstore
texts = [doc["text"] for doc in documents]
metadatas = [doc["metadata"] for doc in documents]
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metadatas)

# Save FAISS index
vectorstore.save_local("db/faiss")


In [ ]:
!zip db.zip "/content/drive/MyDrive/LLM/db/"

updating: content/drive/MyDrive/LLM/db/ (stored 0%)


RAG - Pipeline

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Step 1: Initialize Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Step 2: Load FAISS Index
vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/LLM/db/faiss",  # Path to your FAISS index # db/faiss in my local
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Step 3: Load Pre-trained LLM
model_name = "EleutherAI/gpt-neo-2.7B"  # Replace with your fine-tuned model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Adjust max_new_tokens for generation
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,  # Use GPU (set to -1 for CPU)
    max_new_tokens=200  # Generate up to 200 tokens
)

# Wrap the Hugging Face pipeline in LangChain's LLM wrapper
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Step 4: Create Retriever
retriever = vectorstore.as_retriever()



<ipython-input-5-7f1e5268b089>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

<ipython-input-5-7f1e5268b089>:34: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:

# Step 5: Create Retrieval-Augmented Generation (RAG) Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Step 6: Test the RAG Pipeline
query = "What are the symptoms of diabetes?"
response = rag_chain.invoke({"query": query})

# Extract the answer and source documents
answer = response["result"]
source_documents = response["source_documents"]

print("Answer:", answer)
print("\nSources:")
for source in source_documents:
    print(source.metadata)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Are gI symptoms in diabetes mellitus associated with both poor glycemic control and diabetic complications? Diabetes mellitus is associated with an increased prevalence of GI symptoms, but the mechanisms underlying symptoms are poorly defined and controversial. We aimed to determine whether there is a relationship between GI symptoms and both diabetic complications and glycemic control. yes

Are phobic symptoms , particularly the fear of blood and injury , associated with poor glycemic control in type I diabetic adults? To investigate the presence of psychiatric disorders and symptoms in type I diabetic patients and to identify those that may influence metabolic control as assessed by GHb levels. yes

Is peripheral vascular disease associated with reduced glycosuria in newly diagnosed type 2 diabetic patients? We exa

Deploy on Hugging Face

In [ ]:
!pip install gradio

In [7]:
import gradio as gr
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS  # Use Chroma if needed
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Step 1: Initialize Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Step 2: Load FAISS Index
vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/LLM/db/faiss",  # Path to your FAISS index
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Step 3: Load Pre-trained LLM
model_name = "EleutherAI/gpt-neo-2.7B"  # Replace with your fine-tuned model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Adjust max_new_tokens for generation
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,  # Use GPU (set to -1 for CPU)
    max_new_tokens=200  # Generate up to 200 tokens
)

# Wrap the Hugging Face pipeline in LangChain's LLM wrapper
llm = HuggingFacePipeline(pipeline=llm_pipeline)
# Step 4: Create Retriever
retriever = vectorstore.as_retriever()

# Create the RAG pipeline
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# Define the Gradio interface
def answer_question(question):
    response = rag_chain.run(question)
    sources = response["source_documents"]
    source_texts = "\n".join([doc.metadata["source"] for doc in sources])
    return response["answer"], source_texts

interface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs=["text", "text"],
    title="RAG-Powered Medical QA System",
    description="Ask a medical question and receive a RAG-generated answer along with source references."
)

if __name__ == "__main__":
    interface.launch()


OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 27.06 MiB is free. Process 8458 has 14.72 GiB memory in use. Of the allocated memory 14.61 GiB is allocated by PyTorch, and 11.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!git remote add origin https://huggingface.co/spaces/satwikreddysripathi/MediRAG_v1